In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Stats
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
merged_df = pd.read_csv("cleaned_data/merged_data.csv")

In [3]:
# Helper Functions

def cliffs_delta(x, y):
    """Cliff’s delta for two independent samples."""
    x = pd.Series(x).dropna().values
    y = pd.Series(y).dropna().values
    n_x, n_y = len(x), len(y)
    if n_x == 0 or n_y == 0:
        return np.nan
    greater = 0
    lesser = 0
    for xi in x:
        greater += np.sum(xi > y)
        lesser += np.sum(xi < y)
    delta = (greater - lesser) / (n_x * n_y)
    return delta

In [4]:
# Inferential Tests — H1 (Vigilance / Latency)

results_h1 = {}

# Kruskal–Wallis across quartiles
if {'bat_landing_to_food','rat_pressure_q'}.issubset(merged_df.columns):
    groups = [merged_df.loc[merged_df['rat_pressure_q']==q, 'bat_landing_to_food'].dropna() for q in ['Q1','Q2','Q3','Q4']]
    if all(len(g) > 0 for g in groups):
        kw_stat, kw_p = stats.kruskal(*groups)
        results_h1['kruskal'] = {'H': kw_stat, 'p': kw_p}
    # Mann–Whitney Q1 vs Q4 + Cliff’s delta
    g1 = groups[0]; g4 = groups[-1]
    if len(g1) > 0 and len(g4) > 0:
        u_stat, u_p = stats.mannwhitneyu(g1, g4, alternative='two-sided')
        delta = cliffs_delta(g1, g4)
        results_h1['mw_Q1_vs_Q4'] = {'U': u_stat, 'p': u_p, 'cliffs_delta': delta}

In [ ]:
# GLM for latency
if 'bat_landing_to_food' in merged_df.columns:
    dfm = merged_df.dropna(subset=['bat_landing_to_food']).copy()
    dfm['log_bltf'] = np.log1p(dfm['bat_landing_to_food'])
    preds = []
    for cand in ['rat_minutes','seconds_after_rat_arrival','hours_after_sunset','food_availability','C(habit)','C(month)']:
        base = cand.replace('C(','').replace(')','')
        if ('C(' in cand and base in dfm.columns) or (cand in dfm.columns):
            preds.append(cand)
    formula = "log_bltf ~ " + " + ".join(preds) if preds else "log_bltf ~ 1"
    model = smf.ols(formula=formula, data=dfm)
    if 'period_id' in dfm.columns:
        res = model.fit(cov_type='cluster', cov_kwds={'groups': dfm['period_id']})
    else:
        res = model.fit()
    results_h1['log_linear_model'] = {'formula': formula, 'summary': res.summary().as_text()}

print("\n=== H1 results ===")
for k, v in results_h1.items():
    if isinstance(v, dict) and 'summary' in v:
        print(f"\n[{k}] formula: {v['formula']}\n")
        print(v['summary'])
    else:
        print(k, v)


=== H1 results ===
kruskal {'H': np.float64(3.903699329107659), 'p': np.float64(0.27205238437444934)}
mw_Q1_vs_Q4 {'U': np.float64(25765.0), 'p': np.float64(0.809298333669778), 'cliffs_delta': np.float64(-0.013024324841984293)}

[log_linear_model] formula: log_bltf ~ rat_minutes + seconds_after_rat_arrival + hours_after_sunset + food_availability + C(habit) + C(month)

                            OLS Regression Results                            
Dep. Variable:               log_bltf   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     35.83
Date:                Sun, 14 Sep 2025   Prob (F-statistic):           3.82e-63
Time:                        17:52:27   Log-Likelihood:                -1268.0
No. Observations:                 906   AIC:                             2578.
Df Residuals:                     885   BIC:                             2

In [6]:
# Inferential Tests — H2 (Risk-taking)
results_h2 = {}

# Chi-squared across rat-pressure quartiles
if {'risk','rat_pressure_q'}.issubset(merged_df.columns):
    ct = pd.crosstab(merged_df['rat_pressure_q'], merged_df['risk'])
    if ct.shape == (4, 2):
        chi2, p, dof, exp = stats.chi2_contingency(ct)
        results_h2['chi2_quartiles'] = {'chi2': chi2, 'p': p, 'dof': dof}
        results_h2['crosstab'] = ct

In [7]:
# Logistic regression for risk
if {'risk','rat_minutes'}.issubset(merged_df.columns):
    dfm = merged_df.dropna(subset=['risk','rat_minutes']).copy()
    preds = []
    for cand in ['rat_minutes','seconds_after_rat_arrival','hours_after_sunset','food_availability','C(habit)','C(month)']:
        base = cand.replace('C(','').replace(')','')
        if ('C(' in cand and base in dfm.columns) or (cand in dfm.columns):
            preds.append(cand)
    formula = "risk ~ " + " + ".join(preds) if preds else "risk ~ 1"
    model = smf.logit(formula=formula, data=dfm)
    if 'period_id' in dfm.columns:
        res = model.fit(disp=False, cov_type='cluster', cov_kwds={'groups': dfm['period_id']})
    else:
        res = model.fit(disp=False)
    # Odds ratios + 95% CI
    or_table = np.exp(res.params)
    conf = np.exp(res.conf_int())
    or_df = pd.DataFrame({'OR': or_table, '2.5%': conf[0], '97.5%': conf[1]})
    results_h2['logit'] = {'formula': formula, 'summary': res.summary().as_text(), 'ORs': or_df}

print("\n=== H2 results ===")
for k, v in results_h2.items():
    if k == 'crosstab':
        print("\n[crosstab]\n", v)
    elif isinstance(v, dict) and 'summary' in v:
        print(f"\n[{k}] formula: {v['formula']}\n")
        print(v['summary'])
        display(v['ORs'])
    else:
        print(k, v)


=== H2 results ===
chi2_quartiles {'chi2': np.float64(3.37905721408916), 'p': np.float64(0.3367900511831335), 'dof': 3}

[crosstab]
 risk              0    1
rat_pressure_q          
Q1              105  125
Q2              129  111
Q3              106  103
Q4              118  109

[logit] formula: risk ~ rat_minutes + seconds_after_rat_arrival + hours_after_sunset + food_availability + C(habit) + C(month)

                           Logit Regression Results                           
Dep. Variable:                   risk   No. Observations:                  906
Model:                          Logit   Df Residuals:                      885
Method:                           MLE   Df Model:                           20
Date:                Sun, 14 Sep 2025   Pseudo R-squ.:                  0.9012
Time:                        17:53:55   Log-Likelihood:                -62.034
converged:                      False   LL-Null:                       -627.94
Covariance Type:              clus

/opt/anaconda3/envs/bat-vs-rat/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


,OR,2.5%,97.5%
Intercept,2.112265e+09,NaN,NaN
C(habit)[T.Unknown],1.061977e-18,1.515991e-59,7.439329e+22
C(habit)[T.bat],1.141961e+10,2.624965e-222,4.967973e+241
C(habit)[T.bat_and_pick],3.952563e+10,1.215419e+10,1.285380e+11
C(habit)[T.bat_and_rat],2.298988e+06,1.116932e+06,4.732023e+06
C(habit)[T.bat_fight],8.616092e+12,4.133406e+12,1.796026e+13
C(habit)[T.fast],5.310561e-17,NaN,NaN
C(habit)[T.pick],4.278053e-17,NaN,NaN
C(habit)[T.pick_bat],1.304926e+09,1.552042e+00,1.097157e+18
C(habit)[T.rat],1.105908e+08,5.530339e+07,2.211496e+08


In [9]:
# Inferential Tests — H3 (Reward / Foraging payoff)
results_h3 = {}

# Chi-squared: reward vs quartiles
if {'reward','rat_pressure_q'}.issubset(merged_df.columns):
    ct_q = pd.crosstab(merged_df['rat_pressure_q'], merged_df['reward'])
    if ct_q.shape == (4, 2):
        chi2, p, dof, exp = stats.chi2_contingency(ct_q)
        results_h3['chi2_reward_quartiles'] = {'chi2': chi2, 'p': p, 'dof': dof}
        results_h3['crosstab_quartiles'] = ct_q

# Chi-squared: reward vs risk
if {'reward','risk'}.issubset(merged_df.columns):
    ct_r = pd.crosstab(merged_df['risk'], merged_df['reward'])
    if ct_r.shape == (2, 2):
        chi2, p, dof, exp = stats.chi2_contingency(ct_r)
        results_h3['chi2_reward_vs_risk'] = {'chi2': chi2, 'p': p, 'dof': dof}
        results_h3['crosstab_reward_vs_risk'] = ct_r

# # Logistic regression for reward (includes risk and rat context)
# if {'reward','rat_minutes'}.issubset(merged_df.columns):
#     dfm = merged_df.dropna(subset=['reward','rat_minutes']).copy()
#     preds = []
#     for cand in ['rat_minutes','risk','seconds_after_rat_arrival','hours_after_sunset','food_availability','C(habit)','C(month)']:
#         base = cand.replace('C(','').replace(')','')
#         if ('C(' in cand and base in dfm.columns) or (cand in dfm.columns):
#             preds.append(cand)
#     formula = "reward ~ " + " + ".join(preds) if preds else "reward ~ 1"
#     model = smf.logit(formula=formula, data=dfm)
#     if 'period_id' in dfm.columns:
#         res = model.fit(disp=False, cov_type='cluster', cov_kwds={'groups': dfm['period_id']})
#     else:
#         res = model.fit(disp=False)
#     # Odds ratios + 95% CI
#     or_table = np.exp(res.params)
#     conf = np.exp(res.conf_int())
#     or_df = pd.DataFrame({'OR': or_table, '2.5%': conf[0], '97.5%': conf[1]})
#     results_h3['logit'] = {'formula': formula, 'summary': res.summary().as_text(), 'ORs': or_df}

print("\n=== H3 results ===")
for k, v in results_h3.items():
    if 'crosstab' in k:
        print(f"\n[{k}]\n", v)
    elif isinstance(v, dict) and 'summary' in v:
        print(f"\n[{k}] formula: {v['formula']}\n")
        print(v['summary'])
        display(v['ORs'])
    else:
        print(k, v)


=== H3 results ===
chi2_reward_quartiles {'chi2': np.float64(1.116464807469838), 'p': np.float64(0.7731009739680361), 'dof': 3}

[crosstab_quartiles]
 reward            0    1
rat_pressure_q          
Q1              114  116
Q2              110  130
Q3               95  114
Q4              103  124
chi2_reward_vs_risk {'chi2': np.float64(351.9386595395892), 'p': np.float64(1.6031431718901113e-78), 'dof': 1}

[crosstab_reward_vs_risk]
 reward    0    1
risk            
0        72  386
1       350   98
